<a href="https://colab.research.google.com/github/chunkitchung/Smart-Cities-Hackathon/blob/dev/Fruit_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://bitbucket.org/ishaanjav/code-and-deploy-custom-tensorflow-lite-model/raw/a4febbfee178324b2083e322cdead7465d6fdf95/fruits.zip

In [ ]:
!unzip fruits.zip

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as pit

In [ ]:
img_height, img_width = 32, 32
batch_size = 20

train_ds = tf.keras.utils.image_dataset_from_directory(
    "fruits/train",
    image_size = (img_height, img_width),
    batch_size = batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "fruits/validation",
    image_size = (img_height, img_width),
    batch_size = batch_size
)
tes_ds = tf.keras.utils.image_dataset_from_directory(
    "fruits/test",
    image_size = (img_height, img_width),
    batch_size = batch_size
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
class_names = ["apple", "banana","orange"]
plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3,3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
model = tf.keras.Sequential(
    [
        #changes each pixel value from being 0 to 255 to just 0 to 1
        #simplifies the learning
        tf.keras.layers.Rescaling(1./255),
        #First convolutional layer
        #apply filters to images to learn cetain features
        #learn fautures as well
        tf.keras.layers.Conv2D(32, 3, activation="relu"),
        #Counting down out feature map by 4 times, Pooling layer
        tf.keras.layers.MaxPooling2D(),
         #applu filters to images to learn cetain features
        tf.keras.layers.Conv2D(32, 3, activation="relu"),
        #Counting down out feature map by 4 times, Pooling layer
        tf.keras.layers.MaxPooling2D(),
         #applu filters to images to learn cetain features
        tf.keras.layers.Conv2D(32, 3, activation="relu"),
        #Counting down out feature map by 4 times, Pooling layer
        tf.keras.layers.MaxPooling2D(),
        #Flattens out
        tf.keras.layers.Flatten(),
        #128 neurons
        tf.keras.layers.Dense(128, activation="relu"),
        #3 neurons one for each class
        tf.keras.layers.Dense(3)
    ]
)

In [ ]:
model.compile(
    optimizer="adam",
    #we are trying to minimize this loss function
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    #receive acuracy
    metrics=['accuracy']
)

In [ ]:
#Train our model
model.fit(
    train_ds,
    #it is evaluated on this data
    validation_data = val_ds,
    #goes over the whole training dataset 3 times
    epochs = 10
)

In [ ]:
model.evaluate(tes_ds)

In [ ]:
import numpy
plt.figure(figsize=(10,10))
for images, labels in tes_ds.take(1):
  #pass a list of 20 images to our model and get classification
  classification = model(images)
  #gives us a likelihood of being from each classification
  print(classification)

In [ ]:
import numpy

plt.figure(figsize=(10,10))
for images, labels in tes_ds.take(1):
  classifications = model(images)
  # print(classifications)
  
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    index = numpy.argmax(classifications[i])
    plt.title("Pred: " + class_names[index] + " | Real: " + class_names[labels[i]])